In [1]:
import ingestor
import extractor2

import os
import tensorflow as tf
import tensorflow.contrib.eager as tfe
# tf.compat.v1.enable_eager_execution()
print(tf.__version__)

import time
import ingestor
import extractor2
import utils
import data
import functools

Tesor Flow Version: 1.14.0 Ingest File
Tesor Flow Version: 1.14.0  Utility File
Tesor Flow Version: 1.14.0 Extactor 2
1.14.0


In [2]:
import data
import utils

# Research notebook - Jeff
* python 3.6.5
* tensorflow 1.14
* conda_tensorflow_p36


## Making a vanilla NN after the extractor

In [85]:
# ingestor
input_fn, params = ingestor.get_input_fn()

INFO:tensorflow:PASSED IN parser is None


In [76]:
features, labels = input_fn()[0], input_fn()[1]

In [78]:
# extractor
features, labels = extractor2.Extractor(params).extractor(features, labels)

In [99]:
def manual_iterator():
    input_fn, params = ingestor.get_input_fn()
    features, labels = input_fn(), input_fn()
    features, labels = extractor2.Extractor(params).extractor(features, labels)
    
    goals_X = features['goal_ids']
    tacid_y = labels['tac_id']
    
    return goals_X, tacid_y

# Vanilla NN

In [45]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

mnist = tf.keras.datasets.mnist

In [46]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
(x_train, y_train), (x_test, y_test)

In [82]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

---

# Running extraction pipeline

In [6]:
import importlib
import extraction_pipeline


importlib.reload(extraction_pipeline)

<module 'extraction_pipeline' from '/home/ec2-user/SageMaker/deepmath/deepmath/deephol/train/extraction_pipeline.py'>

In [ ]:
# run
features, labels = extraction_pipeline.run_extraction_pipeline()

Done extracting data into arrays for goals, thms and tactics
Done tokenizing goals
Done tokenizing hypotheses


# upload them

In [70]:
import pandas as pd
import numpy as np
import sys
import boto3

BUCKET_NAME = 'sagemaker-cs281'

In [82]:
def write_np(array, local_filename='/tmp/temp.npy'): # extension is .npy
    with open(local_filename, 'wb') as f:
        np.save(local_filename, array)
    print('Done writing file locally to {}'.format(local_filename))
    
    return local_filename

In [78]:
def upload_to_s3(filename, object_name):
    s3_client = boto3.client('s3')
    response = s3_client.upload_file(filename, BUCKET_NAME, object_name)
    

In [32]:
# # make arrays
# goals_array = np.array(features['goal_ids'])
# hypotheses_array = np.array(features['goal_asl_ids'])
# labels_array = np.array(labels['tac_id'])

In [119]:
def upload_np_to_s3(array, object_name):    
    # save localy
    local_filename = '/tmp/temp.csv'
    np.savetxt(local_filename, array, delimiter=',')
    
    # s3 upload
    s3_client = boto3.client('s3')
    response = s3_client.upload_file(local_filename, BUCKET_NAME, object_name)

In [121]:
train_objs =  'deephol-data-processed/proofs/human/train'


In [122]:
upload_np_to_s3(array, os.path.join(train_objs, 'label_1.csv'))

In [33]:
print([sys.getsizeof(x) for x in [goals_array, hypotheses_array, labels_array]])

[400112, 896, 896]


In [50]:
a = pd.DataFrame(hypotheses_array)

In [23]:
features_array = np.zeros((1000,1000))
for i,x in enumerate(features['goal_ids']):
    features_array[i] = x

In [61]:
filename

'./test.npy'

In [ ]:
file_name = filename
bucket = 
object_name = 

In [91]:
# Upload the file
sys.getsizeof(features['goal_asl_ids'])

912

In [97]:
import pickle

## tfrecord

In [103]:
def tf_serialize_example(*args):
    tf_string = tf.py_function(
        serialize_example,
        (*args),  # pass these args to the above function.
        tf.string)      # the return type is `tf.string`.
    return tf.reshape(tf_string, ()) # The result is a scalar

In [101]:
labels_datset = tf.data.Dataset.from_tensor_slices(labels['tac_id'])

In [106]:
features_dataset = tf.data.Dataset.from_tensor_slices(
    labels['tac_id'])


In [107]:
serialized_features_dataset = features_dataset.map(tf_serialize_example)
serialized_features_dataset

NameError: name 'serialize_example' is not defined

In [102]:
filename = './test.tfrecord'
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(labels_datset)

TypeError: `dataset` must produce scalar `DT_STRING` tensors whereas it produces shape () and types <dtype: 'int64'>